# Predicting Countries GDP


### Author : Jose Juan Peña Gomez

## Introduction

I was looking for various economic indicators, statistics of countries and I came across UNData Kaggle's dataset provided by Data Science for Good : Kiva Crowdfunding.

The United Nations Statistics Division (UNSD) of the Department of Economic and Social Affairs (DESA) launched a new internet based data service for the global user community. It brings UN statistical databases within easy reach of users through a single entry point (http://data.un.org/). Users can search and download a variety of statistical resources of the UN system.

    https://www.kaggle.com/sudalairajkumar/undata-country-profiles

This dataset contains the following key statistical indicators of the countries:

    1- General Information
    2- Economic Indicators
    3- Social Indicators
    4- Environmental & Infrastructure Indicators

The data is from 2017.

This dataset is composed by two files:

    1. Country_profile_variables.csv : contains the indicator variables of all the countries present in UNData.
    2. kiva_country_profile_variables.csv : contains the indicator variables of the countries present in the Kiva Crowdfunding dataset. (Apart from country Kosovo, all the other countries are present in UNData and hence it is in this dataset as well)
    
So finally I decided that the porpuse of this project is going to be about  building a machine learning model which will be able to predict the GDP of every country.


# Set up the project libraries

In this project we are going to use several tools as Numpy for mathematicals issues, Pandas for arrange the data, sklearn for the ML part and matplotlib for plotting the results.

In [13]:
# Import Section

from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [15]:
# Input data files are available in the "input/" directory.

import os
print(os.listdir("input/"))

['kiva_country_profile_variables.csv', 'country_profile_variables.csv']


# Get the data

For getting the data is necesary to first download the dataset from the webpage written before ( Kaggle ). Then I will set the variable df for work with a dataframe of the dataset for take a look.

In [22]:
df = pd.read_csv('input/country_profile_variables.csv')

# Explore the data

We can take a first look of the first rows of the dataframe with the function head.

We can observe that each row is a country and each country has 50 features that in this case mean 50 economics indicators.

In [20]:
df.head()

,country,Region,Surface area (km2),Population in thousands (2017),"Population density (per km2, 2017)","Sex ratio (m per 100 f, 2017)",GDP: Gross domestic product (million current US$),"GDP growth rate (annual %, const. 2005 prices)",GDP per capita (current US$),Economy: Agriculture (% of GVA),...,Mobile-cellular subscriptions (per 100 inhabitants).1,Individuals using the Internet (per 100 inhabitants),Threatened species (number),Forested area (% of land area),CO2 emission estimates (million tons/tons per capita),"Energy production, primary (Petajoules)",Energy supply per capita (Gigajoules),"Pop. using improved drinking water (urban/rural, %)","Pop. using improved sanitation facilities (urban/rural, %)",Net Official Development Assist. received (% of GNI)
0,Afghanistan,SouthernAsia,652864,35530,54.4,106.3,20270,-2.4,623.2,23.3,...,8.3,42,2.1,9.8/0.3,63,5,78.2/47.0,45.1/27.0,21.43,-99
1,Albania,SouthernEurope,28748,2930,106.9,101.9,11541,2.6,3984.2,22.4,...,63.3,130,28.2,5.7/2.0,84,36,94.9/95.2,95.5/90.2,2.96,-99
2,Algeria,NorthernAfrica,2381741,41318,17.3,102.0,164779,3.8,4154.1,12.2,...,38.2,135,0.8,145.4/3.7,5900,55,84.3/81.8,89.8/82.2,0.05,-99
3,American Samoa,Polynesia,199,56,278.2,103.6,-99,-99,-99.0,-99,...,-99,92,87.9,-99,-99,-99,100.0/100.0,62.5/62.5,-99,-99
4,Andorra,SouthernEurope,468,77,163.8,102.3,2812,0.8,39896.4,0.5,...,96.9,13,34.0,0.5/6.4,1,119,100.0/100.0,100.0/100.0,-99,-99


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 50 columns):
country                                                       229 non-null object
Region                                                        229 non-null object
Surface area (km2)                                            229 non-null object
Population in thousands (2017)                                229 non-null int64
Population density (per km2, 2017)                            229 non-null float64
Sex ratio (m per 100 f, 2017)                                 229 non-null float64
GDP: Gross domestic product (million current US$)             229 non-null int64
GDP growth rate (annual %, const. 2005 prices)                229 non-null object
GDP per capita (current US$)                                  229 non-null float64
Economy: Agriculture (% of GVA)                               229 non-null object
Economy: Industry (% of GVA)                                  229 non-null

Please read about the terms of use and policies here.